<a href="https://colab.research.google.com/github/zie225/-Voting-Bagging-random-forest/blob/master/Finetuning_Mistral_7b_4bit_core_sna.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fine-tune Mistral


This code notebook makes it possible to fine-tune Mistral on a set of literary annotations.


## Import of the model

While it's possible to do it straight with the huggingface import functions, as I use  colab and drive, I prefer to import each file individually (espacially less issue with .cache)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

%cd "mistral"
#%cd "/content/drive/Shareddrives/OPSCI/LLMs/mistral"

Mounted at /content/drive
[Errno 2] No such file or directory: 'mistral'
/content


In [ ]:
!mkdir "mistral-7b-v0.1"
%cd "mistral-7b-v0.1"

!wget https://huggingface.co/mistralai/Mistral-7B-v0.1/resolve/main/added_tokens.json
!wget https://huggingface.co/mistralai/Mistral-7B-v0.1/resolve/main/config.json
!wget https://huggingface.co/mistralai/Mistral-7B-v0.1/resolve/main/generation_config.json
!wget https://huggingface.co/mistralai/Mistral-7B-v0.1/resolve/main/pytorch_model-00001-of-00002.bin
!wget https://huggingface.co/mistralai/Mistral-7B-v0.1/resolve/main/pytorch_model-00002-of-00002.bin
!wget https://huggingface.co/mistralai/Mistral-7B-v0.1/resolve/main/special_tokens_map.json
!wget https://huggingface.co/mistralai/Mistral-7B-v0.1/resolve/main/tokenizer.json
!wget https://huggingface.co/mistralai/Mistral-7B-v0.1/resolve/main/tokenizer.model
!wget https://huggingface.co/mistralai/Mistral-7B-v0.1/resolve/main/tokenizer_config.json
!wget https://huggingface.co/mistralai/Mistral-7B-v0.1/resolve/main/pytorch_model.bin.index.json

%cd ".."

/content/mistral-7b-v0.1
--2023-09-28 06:39:44--  https://huggingface.co/mistralai/Mistral-7B-v0.1/resolve/main/added_tokens.json
Resolving huggingface.co (huggingface.co)... 18.172.134.24, 18.172.134.88, 18.172.134.124, ...
Connecting to huggingface.co (huggingface.co)|18.172.134.24|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 42 [text/plain]
Saving to: ‘added_tokens.json’

added_tokens.json   100%[===================>]      42  --.-KB/s    in 0s      

2023-09-28 06:39:44 (24.7 MB/s) - ‘added_tokens.json’ saved [42/42]

--2023-09-28 06:39:44--  https://huggingface.co/mistralai/Mistral-7B-v0.1/resolve/main/config.json
Resolving huggingface.co (huggingface.co)... 18.172.134.88, 18.172.134.4, 18.172.134.124, ...
Connecting to huggingface.co (huggingface.co)|18.172.134.88|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 571 [text/plain]
Saving to: ‘config.json’

config.json         100%[===================>]     571  --.-KB/s    in 0s

## Install

We connect to the drive

We install transformers and peft separately from the latest version on Github. Otherwhise, you will miss key metadata for Mistral support.

In [ ]:
!pip install git+https://github.com/huggingface/transformers.git
!pip install git+https://github.com/huggingface/peft.git

  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-888s9x9m
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-888s9x9m
  Resolved https://github.com/huggingface/transformers.git to commit ab37b801b14d8b9c3186548e6e118aff623e6aa1
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 24.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 27.3 MB/s eta 0:00:00
  Created wheel for transformers: filename=transformers-4.34.0.dev0-py3-none-any.whl size=7746154 sha256=7b5a95e97713ce1cb8854e09ca0e8e617fca257a87d06ba71e1375866f74d5bb
  Stored in directory: /tmp/pip-ephem-wheel-c

We install the other extensions.

In [ ]:
!pip install -q accelerate bitsandbytes trl guardrail-ml tensorboard
!apt-get -qq install poppler-utils tesseract-ocr
!pip install -q unstructured["local-inference"] pillow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.0/118.0 kB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 44.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 24.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.1/105.1 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 74.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 29.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 23.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 78.2 MB/s eta 0:00:00
Selecting previously unselected package poppler-utils.
(Reading database ... 120895 files and directories current

We load the libraries

In [ ]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
    LlamaTokenizerFast
)
from peft import LoraConfig, PeftModel, get_peft_model
from trl import SFTTrainer

## Parameters

Pour plus de commodité, nous déposons l'ensemble des paramètres ici (à commenter plus en détail ultérieurement)

In [ ]:
# Base parameters
local_rank = -1
per_device_train_batch_size = 4
per_device_eval_batch_size = 1
gradient_accumulation_steps = 4
learning_rate = 2e-4 #I prefer a high lr for analytical llm, but to each its own
max_grad_norm = 0.3
weight_decay = 0.001
lora_alpha = 16
lora_dropout = 0.1
lora_r = 64
group_by_length = True
max_seq_length = 1024 #Enough for the default text size.

# The name of Mistral model
model_name = "mistral-7b-v0.1"

# Le name of the new model.
new_model_name = "mistral-7b-sna"

# The number of steps.
# I prefer this to the number of epochs (easier to manage and anticipate the time it takes to finetune)
max_steps = 500

# Activate 4-bit precision base model loading
use_4bit = True

# Activate nested quantization for 4-bit base models
use_nested_quant = False

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4=
bnb_4bit_quant_type = "nf4"

# Number of training epochs
num_train_epochs = 1

# Enable fp16 training
fp16 = True

# Enable bf16 training
bf16 = False

# Use packing dataset creating
packing = False

# Enable gradient checkpointing
gradient_checkpointing = True

# Optimizer to use, original is paged_adamw_32bit
optim = "paged_adamw_32bit"

# Learning rate schedule (constant a bit better than cosine, and has advantage for analysis)
lr_scheduler_type = "constant"

# Fraction of steps to do a warmup for
warmup_ratio = 0.03

# Group sequences into batches with same length (saves memory and speeds up training considerably)
group_by_length = True

# Sauvegarde des steps (permet de faire redémarrer l'entraînement si le fine-tuning ne fonctionne pas)
save_steps = 100

# Log every X updates steps
logging_steps = 1

# The output directory where the model predictions and checkpoints will be written
output_dir = "./mistral-7b-sna"

# Load the entire model on the GPU 0
device_map = {"": 0}

# Visualize training
report_to = "tensorboard"

# Tensorboard logs
tb_log_dir = "./mistral-7b-sna/logs"

Nous téléchargeons le modèle (à n'executer qu'une fois)

Nous l'importons maintenant dans la session avec une configuration spéciale (4bit, bf16) pour accélérer le fine-tuning

In [ ]:
# Load tokenizer and model with QLoRA configuration
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16, you can accelerate training with the argument --bf16")
        print("=" * 80)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map=device_map,
    quantization_config=bnb_config
)

model.config.use_cache = False
model.config.pretraining_tp = 1

NameError: ignored

We load the tokenizer and the peft configuration. Notice you have to specify the target modules as peft is not yet fully updated for Mistral.

Also using the llama fast tokenizer but not sure if this is the best idea…

In [ ]:
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    inference_mode=False,
    task_type="CAUSAL_LM",
    target_modules = ["q_proj", "v_proj"]
)

tokenizer = LlamaTokenizerFast.from_pretrained(model_name, add_eos_token=True, from_slow=True)

# This is the fix for fp16 training
tokenizer.padding_side = "right"

## Dataset preparation

We are going to use our dataset of literary analysis of public domain novels as demo. The analyis relies on a mix of synthetic and manual annotations.

In [ ]:
!wget https://huggingface.co/datasets/Pclanglais/Astree/resolve/main/brahe_instructions.json

We load the data in a custom format

In [ ]:
from datasets import load_dataset

def format_custom(sample):
    instruction = f"<s>Text: {sample['full_text']} \n\n### Analysis:\n\n"
    context = None
    response = f"{sample['analysis']}"
    # join all the parts together
    prompt = "".join([i for i in [instruction, context, response] if i is not None])
    print(prompt)
    return prompt

# template dataset to add prompt to each sample
def template_dataset(sample):
    sample["text"] = f"{format_custom(sample)}{tokenizer.eos_token}"
    return sample

# Loadng the dataset.
data_files = {"train": "brahe_instructions.json"}
dataset = load_dataset("json", data_files=data_files, split="train")

#Transformation du dataset pour utiliser le format guanaco
dataset = dataset.map(template_dataset, remove_columns=list(dataset.features))
dataset

In [ ]:
dataset[40]

## Fine-tuning

Nous lançons l'entraînement :

In [ ]:
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
torch.cuda.empty_cache()

training_arguments = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to="tensorboard"
)

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=packing
)

trainer.train()
#trainer.train(resume_from_checkpoint=True)

Map:   0%|          | 0/3989 [00:00<?, ? examples/s]

Step,Training Loss
1,2.290900
2,2.447600
3,2.149100
4,2.351100
5,2.289800
6,2.286000
7,2.070600
8,2.087700
9,2.116800
10,1.935400


TrainOutput(global_step=500, training_loss=1.152424260020256, metrics={'train_runtime': 1078.6347, 'train_samples_per_second': 7.417, 'train_steps_per_second': 0.464, 'total_flos': 6.189654274638643e+16, 'train_loss': 1.152424260020256, 'epoch': 2.0})

We save the model

In [ ]:
model_to_save = trainer.model.module if hasattr(trainer.model, 'module') else trainer.model  # Take care of distributed/parallel training
model_to_save.save_pretrained(new_model_name)

We merge the model and the LORA to get inference speed up.

In [ ]:
del model
torch.cuda.empty_cache()

from peft import AutoPeftModelForCausalLM

model = AutoPeftModelForCausalLM.from_pretrained(new_model_name, device_map="auto", torch_dtype=torch.bfloat16)
model = model.merge_and_unload()

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
output_merged_dir = os.path.join(new_model_name, "final_merged_checkpoint")
model.save_pretrained(output_merged_dir, safe_serialization=True)

We export the tokenizer files.

In [ ]:
!cp "mistral-7b-v0.1/tokenizer.json" "mistral-7b-sna/final_merged_checkpoint/tokenizer.json"
!cp "mistral-7b-v0.1/tokenizer.model" "mistral-7b-sna/final_merged_checkpoint/tokenizer.model"
!cp "mistral-7b-v0.1/tokenizer_config.json" "mistral-7b-sna/final_merged_checkpoint/tokenizer_config.json"
!cp "mistral-7b-v0.1/special_tokens_map.json" "mistral-7b-sna/final_merged_checkpoint/special_tokens_map.json"

## Inference

To do later: not working for now (but you're free to debug). You may need to delete the runtime to free the memory (since we will use a different implementation), especially if you are on the free colab.

In [ ]:
!pip install git+https://github.com/huggingface/transformers.git

  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-8tptib8j
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-8tptib8j
  Resolved https://github.com/huggingface/transformers.git to commit ab37b801b14d8b9c3186548e6e118aff623e6aa1
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 75.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 80.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 34.5 MB/s eta 0:00:00
  Created wheel for transformers: filename=transformers-4.34.0.dev0-py3-none-any.whl size=7746154 sha256=2c9b056d9cbb77919cf56a0f4a76f371e50786c44bb26bffac2a6ab952b1cfd4
  Stored in directory: /tmp/pip-ephem-wheel-c

In [ ]:
!pip install git+https://github.com/mistralai/vllm-release@add-mistral

  Cloning https://github.com/mistralai/vllm-release (to revision add-mistral) to /tmp/pip-req-build-o_gek8id
  Running command git clone --filter=blob:none --quiet https://github.com/mistralai/vllm-release /tmp/pip-req-build-o_gek8id
  Running command git checkout -b add-mistral --track origin/add-mistral
  Switched to a new branch 'add-mistral'
  Branch 'add-mistral' set up to track remote branch 'add-mistral' from 'origin'.
  Resolved https://github.com/mistralai/vllm-release to commit 9c7a259dc4172d358ff16806df410c56c9bd9acf
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached ninja-1.11.1-py2.py3-none-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (145 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 MB 24.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 76.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.6/211.6 MB 5.1 MB/s eta 0

In [ ]:
from vllm import LLM, SamplingParams
import os

In [ ]:
new_model_name = "mistral-7b-sna"

In [ ]:
output_merged_dir = os.path.join(new_model_name, "final_merged_checkpoint")
llm = LLM(output_merged_dir)

INFO 09-27 23:16:24 llm_engine.py:72] Initializing an LLM engine with config: model='mistral-7b-sna/final_merged_checkpoint', tokenizer='mistral-7b-sna/final_merged_checkpoint', tokenizer_mode=auto, revision=None, trust_remote_code=False, dtype=torch.bfloat16, download_dir=None, load_format=auto, tensor_parallel_size=1, quantization=None, seed=0)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


INFO 09-27 23:18:37 llm_engine.py:204] # GPU blocks: 10954, # CPU blocks: 2048


In [ ]:
sampling_params = SamplingParams(temperature=0.2, top_p=0.95, max_tokens=500)

In [ ]:
prompts = ["""Text: For a long time I used to go to bed early. Sometimes, when I had put out my candle, my eyes would close so quickly that I had not even time to say "I'm going to sleep." And half an hour later the thought that it was time to go to sleep would awaken me; I would try to put away the book which, I imagined, was still in my hands, and to blow out the light; I had been thinking all the time, while I was asleep, of what I had just been reading, but my thoughts had run into a channel of their own, until I myself seemed actually to have become the subject of my book: a church, a quartet, the rivalry between François I and Charles V. This impression would persist for some moments after I was awake; it did not disturb my mind, but it lay like scales upon my eyes and prevented them from registering the fact that the candle was no longer burning. Then it would begin to seem unintelligible, as the thoughts of a former existence must be to a reincarnate spirit; the subject of my book would separate itself from me, leaving me free to choose whether I would form part of it or no; and at the same time my sight would return and I would be astonished to find myself in a state of darkness, pleasant and restful enough for the eyes, and even more, perhaps, for my mind, to which it appeared incomprehensible, without a cause, a matter dark indeed. \n\n### Analysis:\n\n"""]

In [ ]:
outputs = llm.generate(prompts, sampling_params)

Processed prompts: 100%|██████████| 1/1 [00:05<00:00,  5.64s/it]


In [ ]:
outputs